# Footstep Control
This is a implementation of the paper Robust Footstep Planning and LQR Control for Dynamic Quadrupedal Locomotion
 [link](https://arxiv.org/abs/2010.12326) .

## Imports

In [ ]:
import mujoco
import time
import mujoco.viewer
import jax.numpy as jnp
from scipy.optimize import minimize


## Experiments

In [ ]:
# trotting speed

def trotting_speed():
    robot = mujoco.MjModel.from_xml_path('./anybotics_anymal_b/scene.xml')
    robot_data = mujoco.MjData(robot)

    gravity = abs(robot.opt.gravity[2])
 
    """ MPC parameter """
    N = 3
    Ts = 0.3
    Q = 1000  
    R = 1 
    s =  1 #for left support
    ry = 0 #for trotting gait

    """ Model parameters """
    delta_theta = 0.4 # send by user
    theta_zero = 0.56 
    r = 0.41
    # z_CoM = 0.042
    t = 2.5e-3 
    d = 0.5 #TODO: obtain this by simulation

    """ init sim """
    SIM_DURATION = 5 #seconds
    FREQUENCY = 400 #hz
    dt = 1 / FREQUENCY # 2.5ms
    lipm_model = Model(gravity, robot_data, d)
    mpc = MPC(Q, R, lipm_model)

    last_time = robot_data.time
    mujoco.mj_resetData(robot, robot_data)

    #TODO: check correct simulation workflow
    while robot_data.time < SIM_DURATION:
        t0 = lipm_model.compute_t0(robot_data, Ts)

        lipm_model.step(t)

        opt_px, opt_py = mpc.optimize(N, lipm_model.xCoM, lipm_model.yCoM,  
                                          t0, Ts, lipm_model.px, lipm_model.py, 
                                          ry, s)

        print(f"Next ZMP optimal: {opt_px[0]}, {opt_py[0]}")

        # compute the desired zmp on foot
        desired_footHolds = lipm_model.compute_footHold(r, theta_zero, delta_theta, opt_px[0], opt_py[0])

        print("desired footHolds:", desired_footHolds)

        #simulate physics step
        lipm_model.update_state(robot_data, opt_px[0], opt_py[0])

        mujoco.mj_step(robot, robot_data)

        delta_time = robot_data.time - last_time

        if delta_time < dt:
            time.sleep(dt - delta_time)

        last_time = robot_data.time

if __name__ == '__main__':
    trotting_speed()


In [ ]:
# Playground

def playground():
    robot = mujoco.MjModel.from_xml_path('./anybotics_anymal_b/scene.xml')
    robot_data = mujoco.MjData(robot)
    
    sensor_data = {
        'LF_sensor': 0,
        'RF_sensor': 0,
        'LH_sensor': 0,
        'RH_sensor': 0,
    }
    
    with mujoco.viewer.launch_passive(
        model=robot, data=robot_data, show_left_ui=False, show_right_ui=False
    ) as viewer:
        mujoco.mjv_defaultFreeCamera(robot, viewer.cam)

        while viewer.is_running():
            sensor_data['LF_sensor'] = robot_data.sensor('LF_sensor').data.copy()
            sensor_data['RF_sensor'] = robot_data.sensor('RF_sensor').data.copy()
            sensor_data['LH_sensor'] = robot_data.sensor('LH_sensor').data.copy()
            sensor_data['RH_sensor'] = robot_data.sensor('RH_sensor').data.copy()

            print('sensor_data', sensor_data)

            mujoco.mj_step(robot, robot_data)

            viewer.sync()


if __name__ == "__main__":
    playground()


## Model

In [ ]:
class Model():
    def __init__(self, g: float, data: mujoco.MjData, d: float):
        self.w = jnp.sqrt(g/data.subtree_com[1][2])
        self.xCoM = jnp.array([data.subtree_com[1][0], data.subtree_linvel[1][0]])
        self.yCoM = jnp.array([data.subtree_com[1][1], data.subtree_linvel[1][1]])
        self.px = self.xCoM[0]
        self.py = self.yCoM[0]
        self.d = d

        self.leg_states = {
            'LF_sensor': 'stance',
            'RF_sensor': 'stance',
            'LH_sensor': 'stance',
            'RH_sensor': 'stance',
        }

        self.swing_start_time = {
            'LF_sensor': 0,
            'RF_sensor': 0,
            'LH_sensor': 0,
            'RH_sensor': 0,
        }

    def step(self, t):
        """ 
        Update model state at time t base on the model dynamics 
        x_CoM = A(t) * x_CoM + B(t) * px
        y_CoM = A(t) * y_CoM + B(t) * py

        where px and py are the ZMP positions in x and y respectively.
        and A(t) and B(t) are the matrices that define the model dynamics.

        Args:
            t(float): time step
        """
        self.xCoM = self.A(t) @ self.xCoM + self.B(t) * self.px 
        self.yCoM = self.A(t) @ self.yCoM + self.B(t) * self.py 

    def update_state(self, data, px0, py0):
        """get current state of the model from mujoco data"""
        self.xCoM = jnp.array([data.subtree_com[1][0], data.subtree_linvel[1][0]])
        self.yCoM = jnp.array([data.subtree_com[1][1], data.subtree_linvel[1][1]])
        self.px = px0
        self.py = py0

    def A(self, t):
        """A Matrix of model dynamics"""
        return jnp.array([
            [jnp.cosh(self.w * t), (1/self.w) * jnp.sinh(self.w * t)],
            [self.w * jnp.sinh(self.w * t), jnp.cosh(self.w * t)]
        ])

    def B(self, t):
        """B Matrix of model dynamics"""
        return jnp.array([
            1 - jnp.cosh(self.w * t),
            -self.w * jnp.sinh(self.w * t)
        ])

    def get_kinematic_constraints(self, p0, p_seq):
        """
        Get the kinematics constraints to the ZMP positions for the model

        Args:
            p_next: represent the vector of futures ZMP positions
            p0: Current ZMP position
            d: Maximum reach of the legs from the support point.

        Returns:
            A list of constraints
        """
        min = p0 - self.d
        max = p0 + self.d

        N = p_seq.shape[0]

        M = jnp.zeros((2*N, N))
        for i in range(N):
            M[2*i, i] = 1
            M[2*i+1, i] = -1
            if i > 0:
                M[2*i-1, i-1] = -1

        lower_bound = jnp.concatenate([min * jnp.ones(1), -self.d * jnp.ones(N - 1)])
        upper_bound = jnp.concatenate([max * jnp.ones(1), -self.d * jnp.ones(N - 1)])
        b = jnp.concatenate([lower_bound, upper_bound])

        A = jnp.vstack([M, -M])
        constraints = { 'type': 'ineq', 'fun': lambda x: b - A @ x }
 
        return [constraints]

    def compute_t0(self, data, Ts):
        """
        Compute t0 which is the remaining period of the current swing phase

        Args:
            data(mjData): mujoco data
            Ts(float): time step

        Returns:
            t0(float): remaining time of the current swing phase
        """
       
        leg_sensors = ['LF_sensor', 'RF_sensor', 'LH_sensor', 'RH_sensor']
        smallest_t0 = Ts

        for leg in leg_sensors:
            sensor_value = data.sensor(leg).data.copy()   

            if self.leg_states[leg] == "stance" and sensor_value < 10: #threshold
                self.leg_states[leg] =  "swing"
                self.swing_start_time[leg] = data.time
                print(leg, "start swing")

            elif self.leg_states[leg] == "swing" and sensor_value > 10: 
                elapsed_time = data.time - self.swing_start_time[leg]
                t0 = max(0, Ts, Ts - elapsed_time)
                smallest_t0 = min(smallest_t0, t0)
                
        return smallest_t0
        
    def compute_footHold(self, r, theta_zero, delta_theta, opt_px, opt_py):
        """ 
            Compute the desire zmp for each footHold

            Args:
            r (float): fixed value the distance from the ZMP to each foot location.
                       theta_zero (float): constant angle of the default configuration.

            delta_theta (float): offset angle to position the feet slightly 
                                 to the sides, command send by the user. 

            opt_px (float): desired zmp in x.
            opt_py (float): desired zmp in y.

            returns:

            desired_lf (list): desired LF position
            desired_rf (list): desired RF position
            desired_lh (list): desired LH position
            desired_Rf (list): desired RH position
        """

        desired_zmp_point = jnp.array([opt_px, opt_py])

        desired_lf = None
        desired_rf = None
        desired_lh = None
        desired_rh = None

        if (self.leg_states['LF_sensor'] == "swing"):
            lf_rotation = jnp.array([jnp.cos(theta_zero + delta_theta), jnp.sin(theta_zero + delta_theta)])
            desired_lf = desired_zmp_point + r * lf_rotation

        if (self.leg_states['RF_sensor'] == "swing"):
            rf_rotation = jnp.array([jnp.cos(theta_zero - delta_theta), -jnp.sin(theta_zero - delta_theta)])
            desired_rf = desired_zmp_point + r * rf_rotation

        if (self.leg_states['LH_sensor'] == "swing"):
            lh_rotation = jnp.array([-jnp.cos(theta_zero - delta_theta), jnp.sin(theta_zero - delta_theta)])
            desired_lh = desired_zmp_point + r * lh_rotation
    
        if (self.leg_states['RH_sensor'] == "swing"):
            rh_rotation = jnp.array([-jnp.cos(theta_zero + delta_theta), -jnp.sin(theta_zero + delta_theta)])
            desired_rh = desired_zmp_point + r * rh_rotation

        return {
            'LF': desired_lf,
            'RF': desired_rf,
            'LH': desired_lh,
            'RH': desired_rh,
        }

## Planner

In [ ]:
# MPC
class MPC():
    def __init__(self, Q, R, model: Model):
        self.Q = Q
        self.R = R
        self.model = model

    def compute_cost(self, N, xCoM_states, yCoM_states, px, py, xCoM_d, yCoM_d, ry=0, s=0):
        """ 
        Compute cost function for x and y 

        Args:
            N(int): number of steps
            xCoM_states(list): velocity of CoM on x
            yCoM_states(list): velocity of CoM on y
            xCoM_d(list): desire velocity on x
            yCoM_d(list): desire velocity on y
            px(float): ZMP position on x for N steps
            py(float): ZMP position on y for N steps
            ry(float): Distance between the left and right ZMPs
            s(int): is the support phase, s=1 for left support and s=-1 for right support

        Returns:
            Total cost of x and y directions
        """

        cost_x = 0
        cost_y = 0
        
        for i in range(1, N+1):
            cost_x += 0.5 * (self.Q * (xCoM_states[i][1] - xCoM_d)**2 + self.R * (px[i] - px[i-1])**2)
            cost_y += 0.5 * (self.Q * (yCoM_states[i][1] - yCoM_d)**2 + self.R * (py[i] - py[i-1] - s*(-1)**i * ry)**2)

        return cost_x + cost_y

    def compute_CoM_states(self, N, px, py, t0, Ts):
        """
        constraints = [A @ p_seq <= b]
        constraints = [A @ p_seq <= b]
        Compute the future CoM for N steps using the model

        Args:
            N(int): number of steps
            px(float): ZMP position on x for N steps
            py(float): ZMP position on y for N steps
            t0(float): remaining period of the current swing phase.
            Ts(float): fixed swing duration

        Returns:
            a list of CoM for x and y
        """     

        xCoM_states = []
        yCoM_states = []

        self.model.px = px[0]
        self.model.py = py[0]
        self.model.step(t0)

        for i in range(1, N+1):
            self.model.px = px[i]
            self.model.py = py[i]
            self.model.step(Ts)
    
            xCoM_states.append(self.model.xCoM)
            yCoM_states.append(self.model.yCoM)

        return xCoM_states, yCoM_states

    def optimize(self, N, xCoM_d, yCoM_d, t0, Ts, px0, py0, ry=0, s=0):
        """ 
        Find ZMP using Quadratic Programming

        Args:
            N(int): number of steps
            xCoM_d(list): desire CoM velocity on x
            yCoM_d(list): desire CoM velocity on y
            ry(float): Distance between the left and right ZMPs
            s(int): is the support phase, s=1 for left support and s=-1 for right support
            kinematic_limits(list): limits of the model for the point support
            t0(float): remaining period of the current swing phase.
            Ts(float): fixed swing duration

        Returns:
            ZMP in x and y
        """

        initial_guess = jnp.zeros(2*N)

        constraints = [
            {'type': 'ineq', 
             'fun': lambda zmp_positions: self.model.get_kinematic_constraints(px0, zmp_positions[:N])[0] 
            },
            {'type': 'ineq', 
             'fun': lambda zmp_positions: self.model.get_kinematic_constraints(py0, zmp_positions[N:])[0] 
            }
        ]

        bounds = [(-jnp.inf, jnp.inf)] * 2 * N

        def objective_function(zmp_positions):
            px = zmp_positions[:N]
            py = zmp_positions[N:]

            xCoM_states, yCoM_states = self.compute_CoM_states(N, px, py, t0, Ts)
            return self.compute_cost(N, xCoM_states, yCoM_states, px, py, xCoM_d, yCoM_d, ry, s)
        
        result = minimize(objective_function, initial_guess, method='SLSQP', bounds=bounds, constraints=constraints)

        opt_px = result.x[:N]
        opt_py = result.x[N:]


        return opt_px, opt_py

## Controller

## Trajectory generation